# Ocorrências aeronáuticas

Este notebook tem como objetivo apresentar a análise dos dados de ocorrências aeronáuticas no Brasil. Na análise, será possível indentificarmos as ocorrências mais frenquentes e o tipo; quantificá-las e descobrir a média por ano, estado, município e região; 
quantificá-las por operador, operação e fase de operacao, classificar por modelo, danos e fabricante das aeronaves e se faz parte do Plano de Supervisão da Segurança Operacional(PSSO). Com isso, será posível auxiliar os orgãos envolvidos e elaborarem e planejarem medidas de segurança de prevenção, como por exemplo, o PSSO. Os dados das ocorrências aeronáuticas são enviadas pela Força Aérea Brasileira por meio do CENIPA para a ANAC acrescido de informações enriquecidas pela própria ANAC.

## Importação das bibliotecas

In [28]:
from pandas import read_csv as rc

## Função de requisição e chamada

In [29]:
def search_occurrences():
    try:
        return rc(
            'https://sistemas.anac.gov.br/dadosabertos/Seguranca%20Operacional/Ocorrencia/V_OCORRENCIA_AMPLA.csv',
            sep=';',
            skiprows=[0],
            index_col=[0],
        )
    except Exception as e: 
        print(e)
        return None

In [30]:
df_occurrences = search_occurrences()

## Limpeza e prepração dos dados

No trecho de código a seguir será realizado a verificação e remoção dos dados duplicados:

In [31]:
# Quantidade de dados duplicados
print('Dados duplicados: %s' % df_occurrences.duplicated().sum())

# Remove dados duplicados
df_occurrences.drop_duplicates(inplace=True)

# Verificando
print('Após a limpeza: %s' % df_occurrences.duplicated().sum())

Dados duplicados: 10
Após a limpeza: 0


Para a realização da análise será necessário eliminar os dados desnecessários, renomear as colunas e formatar os dados.
Serão utilizadas as colunas:
<ul>
    <li>Numero_da_Ocorrencia</li>
    <li>Classificacao_da_Ocorrencia</li>
    <li>Data_da_Ocorrencia</li>
    <li>Municipio</li>
    <li>Regiao</li>
    <li>Tipo_de_Ocorrencia</li>
    <li>Fase_da_Operacao</li>
    <li>Operacao</li>
    <li>Danos_a_Aeronave</li>
    <li>Modelo</li>
    <li>Nome_do_Fabricante</li>
    <li>PSSO</li>
</ul>

Nesta etapa, serão removidas as colunas desnecessárias:

In [32]:
# Colunas que serão usadas
columns_used = [
    'Numero_da_Ocorrencia', 
    'Classificacao_da_Ocorrencia', 
    'Data_da_Ocorrencia', 
    'Municipio', 
    'Regiao',
    'Operador',
    'Tipo_de_Ocorrencia',
    'Fase_da_Operacao',
    'Operacao',
    'Danos_a_Aeronave',
    'Modelo',
    'Nome_do_Fabricante',
    'PSSO'
]

# Obtendo todas as colunas
all_columns = df_occurrences.columns

# Obtendo a diferença de colunas, que serão removidas
removed_columns = set(all_columns).difference(set(columns_used))

# Removendo colunas desnecessárias
df_occurrences.drop(columns=removed_columns, inplace=True)

A seguir, será feito a renomeação das colunas, utilizando o padrão snake case:

In [33]:
# Função para renomear colunas
rename_column = lambda column: column.lower()

# Renomeando colunas
df_occurrences.rename(columns=rename_column, inplace=True)

# Renomeando index
df_occurrences.index.rename('numero_da_ocorrencia', inplace=True)

Verificando o tipo dos dados de cada coluna:

In [34]:
df_occurrences.dtypes

classificacao_da_ocorrencia    object
data_da_ocorrencia             object
municipio                      object
regiao                         object
operador                       object
tipo_de_ocorrencia             object
fase_da_operacao               object
operacao                       object
danos_a_aeronave               object
modelo                         object
nome_do_fabricante             object
psso                           object
dtype: object

Verificando a existencia de valores ausentes:

In [35]:
# Criando uma função lambda para reutilizá-la
check_na = lambda df: df.isnull().sum()

# Verificando a quantidade de valores ausentes
check_na(df_occurrences)

classificacao_da_ocorrencia       2
data_da_ocorrencia                0
municipio                      3698
regiao                          753
operador                        109
tipo_de_ocorrencia             3019
fase_da_operacao               3391
operacao                        124
danos_a_aeronave               3747
modelo                          403
nome_do_fabricante              550
psso                             53
dtype: int64

Realizando a substituíção dos valores ausentes:

In [36]:
# Substituindo os valores ausentes
df_occurrences['classificacao_da_ocorrencia'].fillna('Não registrado', inplace=True)
df_occurrences['municipio'].fillna('Não registrado', inplace=True)
df_occurrences['regiao'].fillna('Não registrado', inplace=True)
df_occurrences['operador'].fillna('Não registrado', inplace=True)
df_occurrences['tipo_de_ocorrencia'].fillna('Não registrada', inplace=True)
df_occurrences['fase_da_operacao'].fillna('Não registrada', inplace=True)
df_occurrences['operacao'].fillna('Não registrada', inplace=True)
df_occurrences['danos_a_aeronave'].fillna('Não registrado', inplace=True)
df_occurrences['modelo'].fillna('Não registrado', inplace=True)
df_occurrences['nome_do_fabricante'].fillna('Não registrado', inplace=True)
df_occurrences['psso'].fillna('Não registrado', inplace=True)

# Verificando se ainda há valores ausentes
check_na(df_occurrences)

classificacao_da_ocorrencia    0
data_da_ocorrencia             0
municipio                      0
regiao                         0
operador                       0
tipo_de_ocorrencia             0
fase_da_operacao               0
operacao                       0
danos_a_aeronave               0
modelo                         0
nome_do_fabricante             0
psso                           0
dtype: int64

No trecho a seguir, será feito a atualização dos valores da coluna **psso**. Os valores **verdadeiro** serão substituídos por **Sim** e os valores **falso** por **Não**:

In [37]:
# Atualizando os valores da coluna psso
df_occurrences['psso'] = df_occurrences['psso'].replace(['verdadeiro'], 'Sim')
df_occurrences['psso'] = df_occurrences['psso'].replace(['falso'], 'Não')

df_occurrences.head()

,classificacao_da_ocorrencia,data_da_ocorrencia,municipio,regiao,operador,tipo_de_ocorrencia,fase_da_operacao,operacao,danos_a_aeronave,modelo,nome_do_fabricante,psso
numero_da_ocorrencia,,,,,,,,,,,,
25043,Incidente,2020-06-17,ALTO ALEGRE,Norte,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,OTHR,Indeterminada,Voo Privado,Nenhum,182K,CESSNA AIRCRAFT,Sim
25081,Incidente,2013-04-13,Brasília,Centro-Oeste,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,Não registrado,Não registrado,Não
25503,Incidente,2020-10-02,CAMPINAS,Sudeste,AZUL LINHAS AEREAS BRASILEIRAS S.A,F-NI,Subida,Voo Regular,Nenhum,ERJ 190-400,EMBRAER,Sim
25623,Incidente,2020-10-07,ARAÇATUBA,Sudeste,JOAO MAURICIO GOTTARDI LOPES,ARC,Pouso,Voo Privado,Leve,180J,CESSNA AIRCRAFT,Sim
25624,Incidente,2020-10-06,SÃO PAULO,Sudeste,JOSE LUIZ CUNHA RODRIGUES,SCF-NP,Arremetida,Voo Privado,Leve,M20J,MOONEY AIRCRAFT,Sim
